# Prepare Data

In [ ]:
# Import libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load data
export_clean = pd.read_csv("../data/Processed/Export_Trade_Cleaned.csv")
import_clean = pd.read_csv("../data/Processed/Import_Trade_Cleaned.csv")

# Clean and reshape
def clean_and_melt(df, value_name):
    df = df.copy()
    df.columns = df.columns.str.strip()
    
    # Remove $ and commas and convert to float
    for col in df.columns[1:]:
        df[col] = df[col].replace('[\$,]', '', regex=True).astype(float)
    
    # Reshape
    df_long = df.melt(id_vars=["Product"], var_name="Year", value_name=value_name)
    df_long["Year"] = df_long["Year"].astype(int)
    return df_long

export_long = clean_and_melt(export_clean, "Export")
import_long = clean_and_melt(import_clean, "Import")

<>:18: SyntaxWarning:

invalid escape sequence '\$'

<>:18: SyntaxWarning:

invalid escape sequence '\$'

/var/folders/4h/1n1m5c3s6vg_drmsgcbr1lz80000gn/T/ipykernel_51057/1868322662.py:18: SyntaxWarning:

invalid escape sequence '\$'



# U.S. Export vs Import Trade (Top 10 Exported Products, 2009–2024)

In [20]:
# Merge & compute ranks
merged = pd.merge(export_long, import_long, on=["Product", "Year"])
merged["Is_All_Merchandise"] = merged["Product"].str.contains("All Merchandise")

# Select top 10 export products per year (excluding All Merchandise)
ranked = merged[~merged["Is_All_Merchandise"]].copy()
ranked["Rank"] = ranked.groupby("Year")["Export"].rank(method="first", ascending=False)

# Select top 10 per year
top_products = ranked.groupby("Year").apply(lambda g: g.nsmallest(10, "Rank")).reset_index(drop=True)

# Bubble size based on export value (scale down if needed)
top_products["Size"] = top_products["Export"] / 1000  # You can adjust divisor to scale

#  Animated Scatter Plot
fig = px.scatter(
    top_products,
    x="Import",
    y="Export",
    color="Product",
    animation_frame="Year",
    size="Size",
    hover_name="Product",
    hover_data={"Rank": True, "Export": True, "Import": True},
    title="U.S. Export vs Import Trade (Top 10 Exported Products, 2009–2024)"
)

# Add 45° trade balance line
max_val = max(top_products["Export"].max(), top_products["Import"].max())
fig.add_trace(go.Scatter(
    x=[0, max_val],
    y=[0, max_val],
    mode='lines',
    line=dict(color='black', dash='dash'),
    name='Trade Balance = 0'
))

# Layout
fig.update_layout(
    xaxis_title="Import Value (Million USD)",
    yaxis_title="Export Value (Million USD)",
    legend_title="Product Category",
    xaxis=dict(range=[0, max_val * 1.05]),
    yaxis=dict(range=[0, max_val * 1.05]),
    height=700
)

fig.show()
fig.write_html("../img/top10_trade_bubble.html")

/var/folders/4h/1n1m5c3s6vg_drmsgcbr1lz80000gn/T/ipykernel_51057/1379603740.py:10: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



# U.S. Export vs Import: All Merchandise (2009–2024)

In [22]:
merged = pd.merge(export_long, import_long, on=["Product", "Year"])
merged["Is_All_Merchandise"] = merged["Product"].str.contains("All Merchandise")
all_merch = merged[merged["Is_All_Merchandise"]].copy()
all_merch["Size"] = 40  # fixed bubble size

# Animated Plot for All Merchandise only
fig = px.scatter(
    all_merch,
    x="Import",
    y="Export",
    animation_frame="Year",
    size="Size",
    hover_name="Product",
    color_discrete_sequence=["gray"],
    text=["All Merchandise"] * len(all_merch),
    title="U.S. Export vs Import: All Merchandise (2009–2024)"
)

# Add 45-degree trade balance line
max_val = max(all_merch["Export"].max(), all_merch["Import"].max())
fig.add_trace(go.Scatter(
    x=[0, max_val],
    y=[0, max_val],
    mode='lines',
    line=dict(color='black', dash='dash'),
    name='Trade Balance = 0'
))

# Layout settings
fig.update_layout(
    xaxis_title="Import Value (Million USD)",
    yaxis_title="Export Value (Million USD)",
    xaxis=dict(range=[0, max_val * 1.05]),
    yaxis=dict(range=[0, max_val * 1.05]),
    height=650,
    showlegend=False
)

fig.show()
fig.write_html("../img/All_Merchandise_bubble.html")

In [24]:
# Filter for All Merchandise
all_merch = merged[merged["Is_All_Merchandise"]].copy()

# Line plot for All Merchandise export and import over time
fig_all = go.Figure()

# Add export line
fig_all.add_trace(go.Scatter(
    x=all_merch["Year"],
    y=all_merch["Export"],
    mode='lines+markers',
    name="Exports",
    line=dict(color="green"),
    marker=dict(size=6),
    hovertemplate="Year: %{x}<br>Export: %{y:,.0f}M<extra></extra>"
))

# Add import line
fig_all.add_trace(go.Scatter(
    x=all_merch["Year"],
    y=all_merch["Import"],
    mode='lines+markers',
    name="Imports",
    line=dict(color="red"),
    marker=dict(size=6),
    hovertemplate="Year: %{x}<br>Import: %{y:,.0f}M<extra></extra>"
))

# Customize layout
fig_all.update_layout(
    title="U.S. Trade Summary for All Merchandise (2009–2024)",
    xaxis_title="Year",
    yaxis_title="Trade Value (Million USD)",
    height=500,
    legend_title="Flow",
    template="plotly_white"
)

fig_all.show()
fig_all.write_html("../img/Import_export_all_trend.html")